In [3]:
!pip install music21

    100% |████████████████████████████████| 18.0MB 2.0MB/s 
  Running setup.py bdist_wheel for music21 ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - done
  Stored in directory: /root/.cache/pip/wheels/53/8b/a6/be1921c60a68f0bea31c6b6a0a7b125badd61294d6a694407f
Successfully built music21


# Importing files from google drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Importing the required libraries

In [0]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
import tensorflow as tf



# Function for training the network


In [0]:
def train_network():
  #Getting the notes of the music
  
  notes = get_notes()
  
  #defining the length of our vocabulary
  
  n_vocab = len(set(notes))
  
  #preparing sequences
  
  network_input, network_output = prepare_sequences(notes, n_vocab)
  
  #Creating the model
  
  model=create_network(network_input, n_vocab)
  
  #training the model
  
  train(model, network_input, network_output)

# Getting the notes from midi files

In [0]:
def get_notes():
  
  notes = []
  
  for file in glob.glob("drive/My Drive/RapMusic/*.mid"):
    
    midi = converter.parse(file)
    
    print("Parsing %s" %file)
    
    parts = instrument.partitionByInstrument(midi)
    
    notes_to_parse = None
    
    # file has instrument parts
    
    if parts: 
      
      notes_to_parse = parts.parts[0].recurse
    
    else: 
    # file has notes in a flat structure
      
      notes_to_parse = midi.flat.notes

  for element in notes_to_parse:
    
    if isinstance(element, note.Note):
      
      notes.append(str(element.pitch))
      
    elif isinstance(element, chord.Chord):
      
      notes.append('.'.join(str(n) for n in element.normalOrder))
            

            
  return notes


     

# Preparing sequences for your model

In [0]:
def prepare_sequences(notes, n_vocab):
  
  sequence_length = 100
  
  pitchnames=sorted(set(item for item in notes))
  
  note_to_int=dict((note,number) for number,note in enumerate(pitchnames))
  
  network_input = []
  
  network_output = []
  
  for i in range(0,len(notes)-sequence_length,1):
    
    sequence_in = notes[i:i+sequence_length]
    
    sequence_out = notes[i + sequence_length]
    
    network_input.append([note_to_int[char] for char in sequence_in])
    
    network_output.append(note_to_int[sequence_out])
    
  n_patterns=len(network_input)
  
  network_input = np.reshape(network_input, (n_patterns,sequence_length,1))
  
  network_input = network_input/float(n_vocab)
  
  network_output=np_utils.to_categorical(network_output)
    
  return (network_input, network_output)
    

# Creating the model

In [0]:
def create_network(network_input, n_vocab):
  
  model=Sequential()
  
  model.add(LSTM(512,input_shape=(network_input.shape[1],network_input.shape[2]),return_sequences=True))
  
  model.add(Dropout(0.4))
  
  model.add(LSTM(512,return_sequences=True))
  
  model.add(Dropout(0.3))
  
  model.add(LSTM(512))
  
  model.add(Dense(256))
  
  model.add(Dropout(0.3))
  
  model.add(Dense(n_vocab))
  
  model.add(Activation('softmax'))
  
  model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
  
  return model


# Training your model

In [16]:
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
    checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=200, batch_size=64, callbacks=callbacks_list)

if __name__ == '__main__':
    train_network()

Parsing drive/My Drive/RapMusic/Whats_My_Name.mid
Parsing drive/My Drive/RapMusic/U_Cant_Touch_This.mid
Parsing drive/My Drive/RapMusic/Woo-Hah.mid
Parsing drive/My Drive/RapMusic/Will_2K.mid
Parsing drive/My Drive/RapMusic/Whoa.mid
Parsing drive/My Drive/RapMusic/Wild_Wild_West.mid
Parsing drive/My Drive/RapMusic/To_All_DJs.mid
Parsing drive/My Drive/RapMusic/Too_Hot.mid
Parsing drive/My Drive/RapMusic/The_Way_I_Am.mid
Parsing drive/My Drive/RapMusic/Under_The_Influence.mid
Parsing drive/My Drive/RapMusic/Sabotage.mid
Parsing drive/My Drive/RapMusic/Ruff_Ryders_Anthem.mid
Parsing drive/My Drive/RapMusic/Still_Dre.mid
Parsing drive/My Drive/RapMusic/The_Shiznit.mid
Parsing drive/My Drive/RapMusic/Summer_Time.mid
Parsing drive/My Drive/RapMusic/The_Real_Slim_Shady.mid
Parsing drive/My Drive/RapMusic/Sunshine.mid
Parsing drive/My Drive/RapMusic/Stan.mid
Parsing drive/My Drive/RapMusic/Still_Dont_Give_A_F_ck.mid
Parsing drive/My Drive/RapMusic/Role_Model.mid
Parsing drive/My Drive/RapMusi

KeyboardInterrupt: ignored